# Occlusion Robustness Evaluation

**Objective:** Compare YOLOv8 vs RT-DETR performance degradation under synthetic occlusions.

**Hypothesis:** RT-DETR (Transformer) should degrade more gracefully than YOLOv8 (CNN) as occlusion increases, because global attention can reason about partial objects better than local convolutions.

**Experiment Design:**
- Fixed weights (no retraining!)
- Same 400 test images with different occlusion levels
- 4 test sets: 0%, 20%, 40%, 60% occlusion
- Same occlusion patterns for both models (seed=42)
- Generate 24 JSON files (2 models × 4 levels × 3 files)

**Runtime:** ~30-40 minutes total on GPU

## 1. Setup & Environment

In [ ]:
# Check if running in Colab
import os
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running in Google Colab")
    # Clone repository if not already present
    if not os.path.exists('/content/Deep_Learning_Gil_Alon'):
        !git clone https://github.com/YOUR_USERNAME/Deep_Learning_Gil_Alon.git
    os.chdir('/content/Deep_Learning_Gil_Alon')
else:
    print("Running locally")
    # Assume we're in notebooks/ directory
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')

print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install dependencies (Colab only)
if IN_COLAB:
    !pip install -q ultralytics roboflow pyyaml pillow

In [ ]:
# Verify GPU availability
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️ WARNING: No GPU detected! Evaluation will be very slow.")

## 2. Download Dataset (Colab Only)

**NOTE:** If running locally, ensure dataset already exists in `data/raw/test/`

In [ ]:
# Set your Roboflow API key here
ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"  # ← REPLACE THIS!

# Dataset version
DATASET_VERSION = 7  # Update if you're using a different version

In [ ]:
if IN_COLAB:
    from roboflow import Roboflow
    
    # Download dataset
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace("gil-alon-ilxmb").project("deep-learning")
    dataset = project.version(DATASET_VERSION).download("yolov8", location="data/raw")
    
    print(f"✓ Dataset downloaded to: {dataset.location}")
else:
    print("⚠️ Running locally - assuming dataset already exists")
    
# Verify test set exists
test_images = list(Path("data/raw/test/images").glob("*.jpg"))
print(f"Found {len(test_images)} test images")

## 3. Verify Prerequisites

Before generating occlusions, ensure:
- ✅ Model weights exist (from Step 3.2 training)
- ✅ Test index exists (from Step 2)
- ✅ Scripts are present

In [ ]:
from pathlib import Path
import json

# Check required files
required_files = {
    "YOLOv8 weights": "models/yolov8n_baseline.pt",
    "RT-DETR weights": "models/rtdetr_baseline.pt",
    "Test index": "data/processed/evaluation/test_index.json",
    "Occlusion script": "scripts/generate_synthetic_occlusions.py",
    "Evaluation script": "scripts/evaluate_baseline.py",
    "Data YAML helper": "scripts/create_data_yaml.py"
}

all_exist = True
for name, path in required_files.items():
    exists = Path(path).exists()
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path}")
    if not exists:
        all_exist = False

if not all_exist:
    print("\n❌ Some required files are missing!")
    print("\nPlease ensure:")
    print("  1. You've trained models in 02_train_models.ipynb")
    print("  2. You've run build_evaluation_index.py (Step 2)")
    print("  3. All scripts are present in scripts/ directory")
else:
    print("\n✓ All prerequisites satisfied!")

## 4. Generate Synthetic Occlusion Test Sets

Create 4 test sets from the same 400 images:
- `level_000/` - Original (0% occlusion) - baseline
- `level_020/` - 20% occlusion per bbox
- `level_040/` - 40% occlusion per bbox
- `level_060/` - 60% occlusion per bbox

**Time:** ~2-5 minutes depending on image sizes

In [ ]:
# Generate occluded test sets
!python scripts/generate_synthetic_occlusions.py \
    --test_index data/processed/evaluation/test_index.json \
    --images_dir data/raw/test/images \
    --labels_dir data/raw/test/labels \
    --output_dir data/synthetic_occlusion \
    --levels 0.0,0.2,0.4,0.6 \
    --seed 42

In [ ]:
# Verify occlusion test sets created
occlusion_levels = [0, 20, 40, 60]
occlusion_manifest_path = Path("data/synthetic_occlusion/occlusion_manifest.json")

if occlusion_manifest_path.exists():
    with open(occlusion_manifest_path, 'r') as f:
        manifest = json.load(f)
    
    print("✓ Synthetic occlusion test sets created!\n")
    print("Levels generated:")
    for level_name, description in manifest['occlusion_levels'].items():
        print(f"  - {level_name}: {description}")
    
    print("\nStatistics:")
    for level_name, stats in manifest['statistics'].items():
        print(f"  {level_name}:")
        print(f"    - Images: {stats['total_images']}")
        print(f"    - Boxes occluded: {stats['total_boxes_occluded']}")
else:
    print("❌ Manifest not found! Occlusion generation may have failed.")

## 5. Create data.yaml for Each Occlusion Level

Each test set needs its own `data.yaml` for Ultralytics validation.

In [ ]:
# Create data.yaml for original test set (0% occlusion)
!python scripts/create_data_yaml.py \
    --dataset_root data/raw \
    --output data/synthetic_occlusion/level_000/data.yaml \
    --absolute

# Copy test images to level_000 (original, no occlusions)
import shutil
level_000_dir = Path("data/synthetic_occlusion/level_000")
level_000_dir.mkdir(parents=True, exist_ok=True)

# Copy original test images and labels
for subdir in ['images', 'labels']:
    src = Path(f"data/raw/test/{subdir}")
    dst = level_000_dir / subdir
    if dst.exists():
        shutil.rmtree(dst)
    shutil.copytree(src, dst)

print("✓ Created level_000 (original test set)")

# Verify all data.yaml files exist
print("\nVerifying data.yaml files:")
for level in [0, 20, 40, 60]:
    level_name = f"level_{level:03d}"
    yaml_path = Path(f"data/synthetic_occlusion/{level_name}/data.yaml")
    status = "✓" if yaml_path.exists() else "✗"
    print(f"{status} {level_name}/data.yaml")

## 6. Evaluate YOLOv8 on All Occlusion Levels

Run inference with **fixed weights** (no training!).

Generates 12 JSON files:
- `yolo_000_run.json`, `yolo_000_metrics.json`, `yolo_000_predictions.json`
- `yolo_020_run.json`, `yolo_020_metrics.json`, `yolo_020_predictions.json`
- `yolo_040_run.json`, `yolo_040_metrics.json`, `yolo_040_predictions.json`
- `yolo_060_run.json`, `yolo_060_metrics.json`, `yolo_060_predictions.json`

**Time:** ~10 minutes total (400 images × 4 levels)

In [ ]:
# Evaluate YOLOv8 on 0% occlusion (baseline)
print("=" * 60)
print("YOLOv8 Evaluation - 0% Occlusion (Baseline)")
print("=" * 60)

!python scripts/evaluate_baseline.py \
    --model yolo \
    --yolo_weights models/yolov8n_baseline.pt \
    --data_yaml data/synthetic_occlusion/level_000/data.yaml \
    --test_index data/processed/evaluation/test_index.json \
    --output_dir evaluation/occlusion_metrics \
    --run_id yolo_000 \
    --device 0

In [ ]:
# Evaluate YOLOv8 on 20% occlusion
print("=" * 60)
print("YOLOv8 Evaluation - 20% Occlusion")
print("=" * 60)

!python scripts/evaluate_baseline.py \
    --model yolo \
    --yolo_weights models/yolov8n_baseline.pt \
    --data_yaml data/synthetic_occlusion/level_020/data.yaml \
    --test_index data/processed/evaluation/test_index.json \
    --output_dir evaluation/occlusion_metrics \
    --run_id yolo_020 \
    --device 0

In [ ]:
# Evaluate YOLOv8 on 40% occlusion
print("=" * 60)
print("YOLOv8 Evaluation - 40% Occlusion")
print("=" * 60)

!python scripts/evaluate_baseline.py \
    --model yolo \
    --yolo_weights models/yolov8n_baseline.pt \
    --data_yaml data/synthetic_occlusion/level_040/data.yaml \
    --test_index data/processed/evaluation/test_index.json \
    --output_dir evaluation/occlusion_metrics \
    --run_id yolo_040 \
    --device 0

In [ ]:
# Evaluate YOLOv8 on 60% occlusion
print("=" * 60)
print("YOLOv8 Evaluation - 60% Occlusion")
print("=" * 60)

!python scripts/evaluate_baseline.py \
    --model yolo \
    --yolo_weights models/yolov8n_baseline.pt \
    --data_yaml data/synthetic_occlusion/level_060/data.yaml \
    --test_index data/processed/evaluation/test_index.json \
    --output_dir evaluation/occlusion_metrics \
    --run_id yolo_060 \
    --device 0

## 7. Evaluate RT-DETR on All Occlusion Levels

Same procedure for RT-DETR.

Generates 12 more JSON files:
- `rtdetr_000_run.json`, `rtdetr_000_metrics.json`, `rtdetr_000_predictions.json`
- `rtdetr_020_run.json`, `rtdetr_020_metrics.json`, `rtdetr_020_predictions.json`
- `rtdetr_040_run.json`, `rtdetr_040_metrics.json`, `rtdetr_040_predictions.json`
- `rtdetr_060_run.json`, `rtdetr_060_metrics.json`, `rtdetr_060_predictions.json`

**Time:** ~10-15 minutes total (RT-DETR is slower than YOLO)

In [ ]:
# Evaluate RT-DETR on 0% occlusion (baseline)
print("=" * 60)
print("RT-DETR Evaluation - 0% Occlusion (Baseline)")
print("=" * 60)

!python scripts/evaluate_baseline.py \
    --model rtdetr \
    --rtdetr_weights models/rtdetr_baseline.pt \
    --data_yaml data/synthetic_occlusion/level_000/data.yaml \
    --test_index data/processed/evaluation/test_index.json \
    --output_dir evaluation/occlusion_metrics \
    --run_id rtdetr_000 \
    --device 0

In [ ]:
# Evaluate RT-DETR on 20% occlusion
print("=" * 60)
print("RT-DETR Evaluation - 20% Occlusion")
print("=" * 60)

!python scripts/evaluate_baseline.py \
    --model rtdetr \
    --rtdetr_weights models/rtdetr_baseline.pt \
    --data_yaml data/synthetic_occlusion/level_020/data.yaml \
    --test_index data/processed/evaluation/test_index.json \
    --output_dir evaluation/occlusion_metrics \
    --run_id rtdetr_020 \
    --device 0

In [ ]:
# Evaluate RT-DETR on 40% occlusion
print("=" * 60)
print("RT-DETR Evaluation - 40% Occlusion")
print("=" * 60)

!python scripts/evaluate_baseline.py \
    --model rtdetr \
    --rtdetr_weights models/rtdetr_baseline.pt \
    --data_yaml data/synthetic_occlusion/level_040/data.yaml \
    --test_index data/processed/evaluation/test_index.json \
    --output_dir evaluation/occlusion_metrics \
    --run_id rtdetr_040 \
    --device 0

In [ ]:
# Evaluate RT-DETR on 60% occlusion
print("=" * 60)
print("RT-DETR Evaluation - 60% Occlusion")
print("=" * 60)

!python scripts/evaluate_baseline.py \
    --model rtdetr \
    --rtdetr_weights models/rtdetr_baseline.pt \
    --data_yaml data/synthetic_occlusion/level_060/data.yaml \
    --test_index data/processed/evaluation/test_index.json \
    --output_dir evaluation/occlusion_metrics \
    --run_id rtdetr_060 \
    --device 0

## 8. Verify All JSON Files Generated

Should have 24 JSON files total:
- 8 `*_run.json` files
- 8 `*_metrics.json` files
- 8 `*_predictions.json` files

In [ ]:
# Verify all JSON files exist
output_dir = Path("evaluation/occlusion_metrics")
models = ['yolo', 'rtdetr']
levels = ['000', '020', '040', '060']
file_types = ['run', 'metrics', 'predictions']

print("Checking generated JSON files:\n")
print("=" * 80)

all_files_exist = True
total_files = 0

for model in models:
    print(f"\n{model.upper()} Evaluations:")
    for level in levels:
        print(f"\n  Level {level} ({int(level)}% occlusion):")
        for file_type in file_types:
            filename = f"{model}_{level}_{file_type}.json"
            filepath = output_dir / filename
            exists = filepath.exists()
            status = "✓" if exists else "✗"
            
            # Get file size if exists
            size_str = ""
            if exists:
                size_kb = filepath.stat().st_size / 1024
                size_str = f"({size_kb:.1f} KB)"
                total_files += 1
            
            print(f"    {status} {filename} {size_str}")
            
            if not exists:
                all_files_exist = False

print("\n" + "=" * 80)
print(f"\nTotal files created: {total_files} / 24")

if all_files_exist:
    print("✓ All 24 JSON files successfully generated!")
else:
    print("\n❌ Some JSON files are missing!")
    print("Check the evaluation outputs above for errors.")

## 9. Compare Performance Across Occlusion Levels

Quick preview of the degradation curves.

In [ ]:
import json
import pandas as pd

# Load all metrics
results = []

for model in models:
    for level in levels:
        metrics_file = output_dir / f"{model}_{level}_metrics.json"
        if metrics_file.exists():
            with open(metrics_file, 'r') as f:
                data = json.load(f)
            
            agg = data['aggregate_metrics']
            results.append({
                'Model': model.upper(),
                'Occlusion': f"{int(level)}%",
                'mAP@50': round(agg['map50'], 3),
                'mAP@50-95': round(agg['map50_95'], 3),
                'Precision': round(agg['precision'], 3),
                'Recall': round(agg['recall'], 3),
                'FPS': round(agg['fps'], 1)
            })

# Create comparison table
df = pd.DataFrame(results)

print("\n" + "=" * 80)
print("OCCLUSION ROBUSTNESS COMPARISON")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)

In [ ]:
# Calculate degradation percentages
print("\nPerformance Degradation (compared to 0% baseline):\n")

for model in ['YOLO', 'RTDETR']:
    model_results = df[df['Model'] == model]
    baseline = model_results[model_results['Occlusion'] == '0%']['mAP@50'].values[0]
    
    print(f"{model}:")
    for _, row in model_results.iterrows():
        if row['Occlusion'] == '0%':
            continue
        degradation = ((baseline - row['mAP@50']) / baseline) * 100
        print(f"  {row['Occlusion']:>4} occlusion: mAP@50 = {row['mAP@50']:.3f} ({degradation:+.1f}% vs baseline)")
    print()

## 10. Download Results to Google Drive (Colab Only)

Save all JSON files for analysis.

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Create backup directory
    backup_dir = Path('/content/drive/MyDrive/Deep_Learning_Occlusion_Results')
    backup_dir.mkdir(parents=True, exist_ok=True)
    
    # Copy all JSON files
    import shutil
    for json_file in output_dir.glob('*.json'):
        shutil.copy(json_file, backup_dir / json_file.name)
    
    print(f"✓ Saved {len(list(output_dir.glob('*.json')))} JSON files to Google Drive")
    print(f"Location: {backup_dir}")
else:
    print("Running locally - JSON files already saved to evaluation/occlusion_metrics/")

## 11. Summary & Next Steps

**What We Just Did:**
1. ✅ Generated 4 synthetic occlusion test sets (0%, 20%, 40%, 60%)
2. ✅ Evaluated YOLOv8 with fixed weights on all 4 levels
3. ✅ Evaluated RT-DETR with fixed weights on all 4 levels
4. ✅ Generated 24 JSON files (2 models × 4 levels × 3 file types)

**Expected Results:**
- Both models should degrade as occlusion increases
- **RT-DETR should degrade LESS** than YOLOv8 at higher occlusion levels
- This validates the hypothesis: Transformers handle occlusion better

**Next Steps:**
1. Analyze degradation curves in detail
2. Compute per-class robustness (which ingredients are most affected?)
3. Visualize predictions on occluded images
4. Write up findings for project report

In [ ]:
print("\n" + "="*80)
print("OCCLUSION ROBUSTNESS EVALUATION COMPLETE!")
print("="*80)
print(f"\nGenerated {total_files} JSON files in: {output_dir}")
print("\nJSON files ready for analysis:")
print("  - Run metadata: Reproducibility info")
print("  - Metrics: mAP, precision, recall, FPS")
print("  - Predictions: Per-image detections for detailed analysis")
print("\nYou can now:")
print("  1. Commit JSON files to GitHub")
print("  2. Analyze degradation curves")
print("  3. Compare CNN vs Transformer robustness")
print("  4. Generate visualizations for report")
print("\n" + "="*80)